In [6]:
import keras
import numpy as np
import pandas as pd
import mlflow
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin 
from mlflow.models import infer_signature
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [16]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep = ";"
)
train, test = train_test_split(
    data,
    test_size=0.2,
    random_state=42,
    shuffle=True
)
x_train = train.drop(
    "quality", axis=1
).values
y_train = train[["quality"]].values.ravel()

x_val = test.drop(
    "quality", axis=1
).values
y_val = test[["quality"]].values.ravel()
x_train, x_test, y_train, y_test = train_test_split(x_train, x_train, test_size=0.2, random_state=42, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((3134, 11), (3134, 11), (784, 11), (784, 11))

In [17]:
signature = infer_signature(
    x_train, y_train
)
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("ANN-Project2")

<Experiment: artifact_location='mlflow-artifacts:/760068880112179797', creation_time=1743573525892, experiment_id='760068880112179797', last_update_time=1743573525892, lifecycle_stage='active', name='ANN-Project2', tags={}>

In [18]:
def train(params, epochs, x_train, y_train, x_test, y_test, x_val, y_val):
    
    mean = np.mean(x_train, axis = 0)
    var = np.var(x_train, axis = 0)
    model = keras.Sequential(
        [
            keras.Input([x_train.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(1)
        ]
    )
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=keras.losses.MeanSquaredError(),
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    with mlflow.start_run(nested=True):
        model.fit(
            x_train,
            y_train,
            epochs=epochs,
            validation_data=(x_val, y_val),
            batch_size=32
        )
        eval = model.evaluate(x_test, y_test, batch_size=32)
        rmse = eval[1]
        mlflow.log_metric("rmse", rmse)
        mlflow.log_param("learning_rate", params["learning_rate"])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", 32)
        mlflow.log_param("optimizer", "Adam")
        mlflow.log_param("loss", "MSE")
        mlflow.log_param("metrics", "RMSE")
        mlflow.tensorflow.log_model(model, "model")
    return {
        "loss":rmse,
        "status":STATUS_OK,
        "model":model
    }

def objective(params):
    result = train(
        params,
        epochs=3,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        x_val=x_val,    
        y_val=y_val,
    )
    return result



In [20]:
space = {
    "learning_rate": hp.loguniform("learning_rate", low=np.log(1e-5), high=np.log(1e-1)),
}

with mlflow.start_run() as run:
    trails = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trails
    )
    best_run = sorted(
        trails.results,
        key = lambda x: x["loss"]
    )[0]
    
    mlflow.log_params(best)
    mlflow.tensorflow.log_model(
        best_run["model"], "model", signature=signature
    )
    

Epoch 1/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 51s 526ms/step - loss: 1978.7511 - root_mean_squared_error: 44.4832
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1847.1556 - root_mean_squared_error: 42.9635 - val_loss: 196.6388 - val_root_mean_squared_error: 14.0228

Epoch 2/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1741.2993 - root_mean_squared_error: 41.7289
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - loss: 1678.6101 - root_mean_squared_error: 40.9704 - val_loss: 195.4256 - val_root_mean_squared_error: 13.9795

Epoch 3/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 1394.2781 - root_mean_squared_error: 37.3400
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - loss: 1664.1766 - root_mean_squared_error: 40.7916 - val_loss: 193.0283 - val_root_mean_squared_error: 13.8935

 1/25 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 1805.5391 - root_mean_squared_error: 42.491

2025/04/02 11:32:19 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.

2025/04/02 11:32:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run kindly-shrimp-968 at: http://127.0.0.1:5000/#/experiments/760068880112179797/runs/21d167e8a3a64b459dfce3741e5bcbaa

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/760068880112179797

Epoch 1/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 25s 267ms/step - loss: 1956.7151 - root_mean_squared_error: 44.2348
47/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1973.5928 - root_mean_squared_error: 44.4245   
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1964.1158 - root_mean_squared_error: 44.3175 - val_loss: 25.3514 - val_root_mean_squared_error: 5.0350

Epoch 2/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1891.7937 - root_mean_squared_error: 43.4948
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - loss: 1747.1650 - root_mean_squared_error: 41.7960 - val_loss: 180.9126 - val_root_mean_squared_error: 13.4504

Epoch 3/3                                   

2025/04/02 11:32:26 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.

2025/04/02 11:32:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run ambitious-stoat-548 at: http://127.0.0.1:5000/#/experiments/760068880112179797/runs/0a6bdd6ff1ae46259d5b1b3d4edf2a4b

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/760068880112179797  

Epoch 1/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 23s 245ms/step - loss: 2106.5317 - root_mean_squared_error: 45.8970
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2073.7346 - root_mean_squared_error: 45.5372 - val_loss: 36.5384 - val_root_mean_squared_error: 6.0447

Epoch 2/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1892.8929 - root_mean_squared_error: 43.5074
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - loss: 2037.2317 - root_mean_squared_error: 45.1330 - val_loss: 33.8811 - val_root_mean_squared_error: 5.8207

Epoch 3/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2103.863

2025/04/02 11:32:32 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.

2025/04/02 11:32:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run debonair-bass-855 at: http://127.0.0.1:5000/#/experiments/760068880112179797/runs/3d153dbeaf224f97ba3ff3d813a1e2c7

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/760068880112179797  

Epoch 1/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 25s 264ms/step - loss: 1915.9988 - root_mean_squared_error: 43.7721
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1997.4395 - root_mean_squared_error: 44.6887 - val_loss: 14.6284 - val_root_mean_squared_error: 3.8247

Epoch 2/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2121.2156 - root_mean_squared_error: 46.0567
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: 1936.0419 - root_mean_squared_error: 43.9966 - val_loss: 16.0050 - val_root_mean_squared_error: 4.0006

Epoch 3/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1923.8381 

2025/04/02 11:32:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.

2025/04/02 11:32:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run intelligent-stag-663 at: http://127.0.0.1:5000/#/experiments/760068880112179797/runs/f65f251964e2475e91d4de2177a90d00

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/760068880112179797  

100%|██████████| 4/4 [00:25<00:00,  6.33s/trial, best loss: 41.544273376464844]
🏃 View run legendary-ape-757 at: http://127.0.0.1:5000/#/experiments/760068880112179797/runs/0b71930124e4440a9f17593977719c19
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/760068880112179797
